In [1]:
# Include necessary packages
library(tidyverse)

# library(mvtnorm)
library(condMVNorm)
library(MomTrunc)
# library(truncnorm)
# library(tmvtnorm)
# library(TTmoment)


library(doParallel)
# library(parallel)



path=getwd()
source(paste0(substring(path, 
                        1, 
                        tail(unlist(gregexpr(pattern ='/',path)),n=1)),
              "MixCenMVReg_EM.R"))

source(paste0(substring(path, 
                        1, 
                        tail(unlist(gregexpr(pattern ='/',path)),n=1)),
              "Util_Func.R"))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: mvtnorm

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Loading required package: parallel



In [2]:
load('true_pars.RData')
load('data_cens_case2_r51n1000.RData')

In [3]:
start=Sys.time()

no_cores=detectCores(logical = TRUE)  
# returns the number of available hardware threads, 
# and if it is FALSE, returns the number of physical cores

cl=makeCluster(10)  
registerDoParallel(cl)  

replicate=length(data_cens_case2)


case2_cens_bias=foreach(i=1:replicate, .errorhandling="pass") %dopar% {
  
    library(condMVNorm)
    library(MomTrunc)
    MixCenMVReg_EM(Y=data_cens_case2[[i]]$Y,
                   X=data_cens_case2[[i]]$X,
                   C=data_cens_case2[[i]]$censorID, 
                   G=3, pie_hat=true_pars$PIE, beta_hat=true_pars$BETA, sigma_hat=true_pars$SIGMA,
                   Max.iter=1000, diff.tol=1e-3, print=FALSE, calc_cov=FALSE)
}
stopCluster(cl)
# save(model1.1,file = "model1.1.RData")
Sys.time()-start

save(case2_cens_bias,file='case2_cens_bias_r51n1000.RData')

Time difference of 38.96994 mins

In [4]:
start=Sys.time()

no_cores=detectCores(logical = TRUE)  
# returns the number of available hardware threads, 
# and if it is FALSE, returns the number of physical cores

cl=makeCluster(10)  
registerDoParallel(cl)  

replicate=length(data_cens_case2)


case2_unce_bias=foreach(i=1:replicate, .errorhandling="pass") %dopar% {
  
    library(condMVNorm)
    library(MomTrunc)
    MixCenMVReg_EM(Y=data_cens_case2[[i]]$Y,
                   X=data_cens_case2[[i]]$X,
                   C=0*data_cens_case2[[i]]$censorID, 
                   G=3, pie_hat=true_pars$PIE, beta_hat=true_pars$BETA, sigma_hat=true_pars$SIGMA,
                   Max.iter=1000, diff.tol=1e-3, print=FALSE, calc_cov=FALSE)
}
stopCluster(cl)
# save(model1.1,file = "model1.1.RData")
Sys.time()-start

save(case2_unce_bias,file='case2_unce_bias_r51n1000.RData')

Time difference of 38.58339 mins

In [5]:
start=Sys.time()

no_cores=detectCores(logical = TRUE)  
# returns the number of available hardware threads, 
# and if it is FALSE, returns the number of physical cores

cl=makeCluster(10)  
registerDoParallel(cl)  

replicate=length(data_cens_case2)


case2_dltc_bias=foreach(i=1:replicate, .errorhandling="pass") %dopar% {
  
    library(condMVNorm)
    library(MomTrunc)
    nocensor=(data_cens_case2[[i]]$censorID[,1]==0 & data_cens_case2[[i]]$censorID[,2]==0)
    
    MixCenMVReg_EM(Y=data_cens_case2[[i]]$Y[nocensor,],
                   X=data_cens_case2[[i]]$X[nocensor,],
                   C=data_cens_case2[[i]]$censorID[nocensor,], 
                   G=3, pie_hat=true_pars$PIE, beta_hat=true_pars$BETA, sigma_hat=true_pars$SIGMA,
                   Max.iter=1000, diff.tol=1e-3, print=FALSE, calc_cov=FALSE)
}
stopCluster(cl)
# save(model1.1,file = "model1.1.RData")
Sys.time()-start

save(case2_dltc_bias,file='case2_dltc_bias_r51n1000.RData')

Time difference of 3.318597 mins